In [ ]:
# Python 3.5.3

import time, random
from selenium import webdriver
from urllib.parse import urlparse, parse_qs

In [ ]:
url = 'https://www.facebook.com/saved'
browser = webdriver.PhantomJS()
# browser = webdriver.Chrome()
browser.get(url)

In [ ]:
browser.find_element_by_css_selector('input#email').send_keys('') # Login email here
browser.find_element_by_css_selector('input#pass').send_keys('') # Login password here
browser.find_element_by_css_selector('button#loginbutton').click()

In [ ]:
# Scroll to the bottom of the page, until we reach the end and the feed stops loading 
previousScroll = browser.execute_script("return document.body.scrollHeight;")

# Try multiple times, in case previous attempt did not load due to network delay 
for j in range(0, 3):
	browser.execute_script("window.scrollTo(0, 0);")
	time.sleep(random.random()) # This probably doesn't need to be random, I just don't want FB to notice I'm scraping
	while True:
		browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
		time.sleep(2 + random.random())
		if previousScroll != browser.execute_script("return document.body.scrollHeight;"):
			previousScroll = browser.execute_script("return document.body.scrollHeight;")
		else:
			break
	print("I think I reached the bottom. Will try to scroll down " + str(2-j) + " more time(s)")
	time.sleep(2 + random.random())

print("Yup, that's it")

In [ ]:
# browser.save_screenshot('now.png')

In [ ]:
# Modified from Oscar Cassetti's code 
# https://www.quora.com/How-can-I-export-all-my-Facebook-saved-read-it-later-links

links = browser.find_elements_by_css_selector('._4bl9._5yjp') # this is the div with the links

for link in links:
    link_url = link.find_element_by_css_selector('a').get_attribute('href')
    # sometimes the url gets dirty FB redirection. Clean it up:
    o = urlparse(link_url)
    try:
        if parse_qs(o.query)['u']:
            link_url = parse_qs(o.query)['u'][0]
    except:
        True # Not an elegant exception handling... but whatever! This only needs to work once.

    link_description = link.text.replace('\n', '|').replace(';', '|') # 1 item per row; semicolon will divide column  
    
    with open('saved.txt', 'a') as f:
        f.write(link_url+' ; '+link_description+'\n')